# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [81]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [82]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.00632,2.31,0.0,6.575,65.2,4.0900,15.3,396.90,4.98,1,0,0,1,0
1,0.02731,7.07,0.0,6.421,78.9,4.9671,17.8,396.90,9.14,1,0,1,0,0
2,0.02729,7.07,0.0,7.185,61.1,4.9671,17.8,392.83,4.03,1,0,1,0,0
3,0.03237,2.18,0.0,6.998,45.8,6.0622,18.7,394.63,2.94,1,0,1,0,0
4,0.06905,2.18,0.0,7.147,54.2,6.0622,18.7,396.90,5.33,1,0,1,0,0


In [83]:
age = boston_features["AGE"]
b = boston_features["B"]
rm = boston_features['RM']
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))
boston_features['RM'] = (rm-min(rm))/(max(rm)-min(rm))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [84]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,0.577505,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,0.547998,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,0.694386,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,0.658555,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,0.687105,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


In [85]:
boston_features.rename(columns={'RAD_(0, 6]': 'RAD_0_6', 'RAD_(6, 24]': 'RAD_6_24', 'TAX_(0, 270]': 'TAX_0_270',
                               'TAX_(270, 360]': 'TAX_270_360', 'TAX_(360, 712]': 'TAX_360_712'}, inplace=True)
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD_0_6,RAD_6_24,TAX_0_270,TAX_270_360,TAX_360_712
0,0.000000,-1.704344,0.0,0.577505,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,0.547998,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,0.694386,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,0.658555,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,0.687105,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [5]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [7]:
boston_target_df = pd.DataFrame(boston.target, columns = ['MEDV'])
boston_target_df.head()
                                

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [15]:
outcome = 'MEDV'
pred_sum = '+'.join(boston_features.columns)
formula = outcome + "~" + pred_sum
boston_whole = pd.concat([boston_features, boston_target_df], axis=1)
boston_whole.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD_0_6,RAD_6_24,TAX_0_270,TAX_270_360,TAX_360_712,MEDV
0,0.000000,-1.704344,0.0,0.577505,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0,24.0
1,0.153211,-0.263239,0.0,0.547998,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0,21.6
2,0.153134,-0.263239,0.0,0.694386,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0,34.7
3,0.171005,-1.778965,0.0,0.658555,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0,33.4
4,0.250315,-1.778965,0.0,0.687105,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0,36.2


In [16]:
formula

'MEDV~CRIM+INDUS+CHAS+RM+AGE+DIS+PTRATIO+B+LSTAT+RAD_0_6+RAD_6_24+TAX_0_270+TAX_270_360+TAX_360_712'

In [17]:
model = ols(formula = formula, data = boston_whole).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Fri, 19 Apr 2019   Prob (F-statistic):          5.08e-153
Time:                        00:04:17   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       9.6013      1.219      7.877      0.000       7.206      11.996
CRIM           -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS          -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS            2.5959      0.796      3.260      0.001       1.032       4.160
RM             13.8125      2.129      6.488      0.000       9.629      17.996
AGE             0.0794      0.352      0.226      0.821      -0.612       0.770
DIS           -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO        -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B               3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT          -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_0_6         4.1316      0.546      7.565      0.000       3.059       5.205
RAD_6_24        5.4696      0.819      6.679      0.000       3.861       7.079
TAX_0_270       4.3357      0.551      7.864      0.000       3.252       5.419
TAX_270_360     3.0759      0.561      5.486      0.000       1.974       4.178
TAX_360_712     2.1896      0.534      4.100      0.000       1.140       3.239
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                     1.81e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.35e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
y = boston.target
linreg = LinearRegression()
linreg.fit(boston_features, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [20]:
linreg.coef_

array([ -1.95380233,  -0.80457549,   2.59586776,  13.81245463,
         0.07939727, -10.09618465,  -1.48666599,   3.8412139 ,
        -5.62879369,  -0.66898159,   0.66898159,   1.13527933,
        -0.12449679,  -1.01078255])

In [21]:
linreg.intercept_

17.602298116498684

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [22]:
boston_features = boston_features.drop(['RAD_6_24', 'TAX_360_712'], axis=1)
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD_0_6,TAX_0_270,TAX_270_360
0,0.000000,-1.704344,0.0,0.577505,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,1
1,0.153211,-0.263239,0.0,0.547998,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,1,0
2,0.153134,-0.263239,0.0,0.694386,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,1,0
3,0.171005,-1.778965,0.0,0.658555,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,1,0
4,0.250315,-1.778965,0.0,0.687105,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,1,0


### Statsmodels

In [27]:
boston_features_int = sm.add_constant(boston_features)
model = sm.OLS(pd.Series(boston.target), boston_features_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Fri, 19 Apr 2019   Prob (F-statistic):          5.08e-153
Time:                        00:25:40   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          17.2605      2.398      7.197      0.000      12.548      21.973
CRIM           -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS          -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS            2.5959      0.796      3.260      0.001       1.032       4.160
RM             13.8125      2.129      6.488      0.000       9.629      17.996
AGE             0.0794      0.352      0.226      0.821      -0.612       0.770
DIS           -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO        -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B               3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT          -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_0_6        -1.3380      0.672     -1.990      0.047      -2.659      -0.017
TAX_0_270       2.1461      0.704      3.047      0.002       0.762       3.530
TAX_270_360     0.8863      0.607      1.459      0.145      -0.307       2.080
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                         30.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Scikit-learn

In [24]:
linreg.fit(boston_features, y)
linreg.coef_

array([ -1.95380233,  -0.80457549,   2.59586776,  13.81245463,
         0.07939727, -10.09618465,  -1.48666599,   3.8412139 ,
        -5.62879369,  -1.33796317,   2.14606188,   0.88628576])

In [25]:
linreg.intercept_

17.260497157052612

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

MEDV changes by -1.49 for every unit of PTRATIO',  where PTRATIO' is the log of PTRATION scaled by Standardization
MEDV changed by 13.81 for every unit of RM', where RM' is RM that was scaled by Min_Max transformation

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [74]:
crimx = .15 #log and minmax
indusx = 6.07 #log and std
rmx = 6.1 # minmax
agex = 33.2 #std
disx = 7.6 #log and minmax
ptratiox = 17 # log and std
bx = 383 # minmax
lstatx = 10.87 #log and std


In [75]:
crimx = np.log(crimx)
indusx = np.log(indusx)
rmx = rmx
agex = agex
disx = np.log(disx)
ptratiox = np.log(ptratiox)
bx = bx
lstatx = np.log(lstatx)

In [76]:
age = boston_features["AGE"]
b = boston_features["B"]
rm = boston_features['RM']
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

In [77]:
crimx = (crimx-min(logcrim))/(max(logcrim)-min(logcrim))
indusx = (indusx-np.mean(logindus))/np.sqrt(np.var(logindus))
rmx = (rmx-min(rm))/(max(rm)-min(rm))
agex = (agex-np.mean(age))/np.sqrt(np.var(age))
disx = (disx-min(logdis))/(max(logdis)-min(logdis))
ptratiox = (ptratiox-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))
bx = (bx-min(b))/(max(b)-min(b))
lstatx = (lstatx-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [80]:
x_test = np.array([crimx, indusx, 1, rmx, agex, disx, ptratiox, bx, lstatx, 0,1,0,1,0])

In [86]:
y = boston.target
linreg = LinearRegression()
linreg.fit(boston_features, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [87]:
coef = linreg.coef_
coef

array([ -1.95380233,  -0.80457549,   2.59586776,  13.81245463,
         0.07939727, -10.09618465,  -1.48666599,   3.8412139 ,
        -5.62879369,  -0.66898159,   0.66898159,   1.13527933,
        -0.12449679,  -1.01078255])

In [88]:
y_eq = x_test*coef
y_eq

array([-0.64774557,  0.36986863,  2.59586776,  6.71964405, -0.09987793,
       -8.10868434,  0.89058793,  3.70658061, -0.14103961, -0.        ,
        0.66898159,  0.        , -0.12449679, -0.        ])

In [89]:
y_int = linreg.intercept_

In [90]:
y_hat = sum(y_eq) + y_int
y_hat

23.431984448909652

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.